In [ ]:
import os
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import shutil

folder_path_exp = '/content/drive/MyDrive/images'
folder_path_age = '/content/drive/MyDrive/utkcropped'

destination_dir_exp = '/content/images'
destination_dir_age = '/content/utkcropped'

shutil.copytree(folder_path_exp, destination_dir_exp)
shutil.copytree(folder_path_age, destination_dir_age)


'/content/utkcropped'

In [10]:
# konstanter
batch_size = 128
picture_size = 48
num_epochs = 48
no_of_classes_exp = 7
no_of_classes_age = 104


In [11]:
# Datagenerering

datagen = ImageDataGenerator(rescale=1./255)

train_set_exp = datagen.flow_from_directory(
    os.path.join(destination_dir_exp, "train"),
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    )

val_set_exp = datagen.flow_from_directory(
    os.path.join(destination_dir_exp, "validation"),
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    )

train_set_age = datagen.flow_from_directory(
    os.path.join(destination_dir_age, "train"),
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    )

val_set_age = datagen.flow_from_directory(
    os.path.join(destination_dir_age, "validation"),
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    )

Found 28835 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Found 23727 images belonging to 104 classes.
Found 23714 images belonging to 104 classes.


In [12]:
# Lager för modellerna

input_layer = Input(shape=(48, 48, 1))

conv1 = Conv2D(64, (3, 3), padding='same', activation='relu')(input_layer)
conv1 = BatchNormalization()(conv1)
conv1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv1 = Dropout(0.25)(conv1)

conv2 = Conv2D(128, (5, 5), padding='same', activation='relu')(conv1)
conv2 = BatchNormalization()(conv2)
conv2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv2 = Dropout(0.4)(conv2)

conv3 = Conv2D(512, (3, 3), padding='same', activation='relu')(conv2)
conv3 = BatchNormalization()(conv3)
conv3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv3 = Dropout(0.25)(conv3)

conv4 = Conv2D(512, (3, 3), padding='same', activation='relu')(conv3)
conv4 = BatchNormalization()(conv4)
conv4 = MaxPooling2D(pool_size=(2, 2))(conv4)
conv4 = Dropout(0.25)(conv4)

flatten_layer = Flatten()(conv4)

fc1 = Dense(256, activation='relu')(flatten_layer)
fc1 = BatchNormalization()(fc1)
fc1 = Dropout(0.25)(fc1)

fc2 = Dense(512, activation='relu')(fc1)
fc2 = BatchNormalization()(fc2)
fc2 = Dropout(0.25)(fc2)

In [13]:
# Output för båda modellerna
expression_output = Dense(no_of_classes_exp, activation='softmax', name='expression_output')(fc2)
age_output = Dense(no_of_classes_age, activation='softmax', name='age_output')(fc2)


In [14]:
# Compilar båda modellerna
expression_model = Model(inputs=input_layer, outputs=expression_output)
age_model = Model(inputs=input_layer, outputs=age_output)

opt_expression = Adam(learning_rate=0.0001)
opt_age = Adam(learning_rate=0.0001)

expression_model.compile(
    optimizer=opt_expression,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

age_model.compile(
    optimizer=opt_age,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)
reduce_learningrate = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, min_delta=0.0001)

callbacks_list = [early_stopping, reduce_learningrate]

In [ ]:
# Träna första modellen
expression_model.fit(
    train_set_exp,
    epochs=num_epochs,
    validation_data=val_set_exp,
    callbacks=callbacks_list
)

Epoch 1/48
226/226 [==============================] - 22s 75ms/step - loss: 2.4203 - accuracy: 0.1993 - val_loss: 2.3609 - val_accuracy: 0.1717 - lr: 1.0000e-04
Epoch 2/48
226/226 [==============================] - 16s 70ms/step - loss: 2.1098 - accuracy: 0.2463 - val_loss: 2.1428 - val_accuracy: 0.2038 - lr: 1.0000e-04
Epoch 3/48
226/226 [==============================] - 16s 72ms/step - loss: 1.9676 - accuracy: 0.2856 - val_loss: 1.9982 - val_accuracy: 0.3054 - lr: 1.0000e-04
Epoch 4/48
226/226 [==============================] - 17s 74ms/step - loss: 1.8604 - accuracy: 0.3202 - val_loss: 1.9165 - val_accuracy: 0.3524 - lr: 1.0000e-04
Epoch 5/48
226/226 [==============================] - 18s 78ms/step - loss: 1.7862 - accuracy: 0.3417 - val_loss: 1.7296 - val_accuracy: 0.3760 - lr: 1.0000e-04
Epoch 6/48
226/226 [==============================] - 16s 72ms/step - loss: 1.7161 - accuracy: 0.3660 - val_loss: 1.7514 - val_accuracy: 0.3902 - lr: 1.0000e-04
Epoch 7/48
226/226 [==============

In [20]:
expression_model.save('exp_model.h5')

In [18]:
# Träna andra modellen

age_model.fit(
    train_set_age,
    epochs=num_epochs,
    validation_data=val_set_age,
    callbacks=callbacks_list
)

Epoch 1/48
186/186 [==============================] - 44s 214ms/step - loss: 4.8211 - accuracy: 0.0661 - val_loss: 4.6871 - val_accuracy: 0.0909 - lr: 1.0000e-04
Epoch 2/48
186/186 [==============================] - 33s 178ms/step - loss: 4.1009 - accuracy: 0.1117 - val_loss: 3.9021 - val_accuracy: 0.1295 - lr: 1.0000e-04
Epoch 3/48
186/186 [==============================] - 35s 186ms/step - loss: 3.7907 - accuracy: 0.1294 - val_loss: 3.4444 - val_accuracy: 0.1618 - lr: 1.0000e-04
Epoch 4/48
186/186 [==============================] - 34s 181ms/step - loss: 3.6206 - accuracy: 0.1366 - val_loss: 3.2236 - val_accuracy: 0.1861 - lr: 1.0000e-04
Epoch 5/48
186/186 [==============================] - 33s 179ms/step - loss: 3.5057 - accuracy: 0.1438 - val_loss: 3.2451 - val_accuracy: 0.1875 - lr: 1.0000e-04
Epoch 6/48
186/186 [==============================] - 33s 176ms/step - loss: 3.4183 - accuracy: 0.1492 - val_loss: 4.3194 - val_accuracy: 0.1179 - lr: 1.0000e-04
Epoch 7/48
186/186 [========

In [19]:
age_model.save('age_model.h5')

In [21]:
# Kombinera modellerna

combined_model = Model(inputs=input_layer, outputs=[expression_output, age_output])

combined_model.save('combined_model.h5')
